In [ ]:
import numpy as np
import json
from astropy.table import Table, vstack


In [ ]:
mSN = [1e9/Table.read('/pscratch/sd/h/hebina/AbacusLBG/ibis_tertiary44/LAE_auto_v2/hod/matter_AbacusSummit_high_c000_ph100_z2.5.fits')['x'].size,
       1e9/Table.read('/pscratch/sd/h/hebina/AbacusLBG/ibis_tertiary44/LAE_auto_v2/hod/matter_AbacusSummit_high_c000_ph100_z3.0.fits')['x'].size]
mSN

# This code manually takes out shot noise 

# $z=3$

In [8]:
zz = 3.
ihod = 1
kpow = 1.25 # manual adjustment to covariance

In [9]:

# ff = open('/pscratch/sd/h/hebina/AbacusLBG/ibis_tertiary44/LAE_auto/lae_clustering_concat_c000_ph100_s_zcv.json')
ff = open('/pscratch/sd/h/hebina/AbacusLBG/ibis_tertiary44/LAE_auto_v2/hod/lae_clustering_c000_ph100_z3.0_s_zcv.json')
# ff = open(self.datfn)
simdata = json.load(ff)
sim_k = np.array(simdata['k'])
k = np.array(simdata['k'])
Nk = len(k)
Nmu = 50
mu = np.linspace(0.,1.,Nmu)
MU = np.tile(mu,Nk)
dmu = list(mu[1:]-mu[:-1])
dmu.append(dmu[-1])
dmu = np.array(dmu)

dk = list(k[1:]-k[:-1])
dk.insert(0,dk[0])
dk = np.array(dk)
k = np.repeat(k,Nmu)
dk = np.repeat(dk,Nmu)
dmu = np.tile(dmu,Nk)

print(simdata['mocks'][ihod]['hod'])
dd = [simdata['mocks'][ihod]['pk0'],simdata['mocks'][ihod]['pk2']]
dd = np.array(dd).T

# take out shot noise
true_SN = 1e9/min(simdata['mocks'][ihod]['nobj'],1e7)

dvec = np.array([np.repeat(dd[:,0],Nmu),np.repeat(dd[:,1],Nmu)])
d_pkmu = dvec[0]+0.5*(3*MU**2-1)*dvec[1]
d_pkmu -= true_SN
print(k.shape,d_pkmu.shape)
Vsurvey = (1e3)**3

prefactor = (2.*np.pi**2.) / (dk*dmu*Vsurvey*k**2.) * k**kpow
Cov = prefactor*2*d_pkmu**2
# constraints = forecast1.compute_wedge(z,kmin=kmin)*forecast1.kmax_constraint(z,kmax_knl)
# constraints *= (forecast1.k > kmin)
knl  = 0.5
kmin = 0.003
kmax = knl


k_constraint = (k>kmin)*(k<kmax)
Cov += 1e-20
Cinv = 1/Cov
Cinv *= k_constraint
# Cinv = np.einsum('i,j,ij->ij',k_constraint,k_constraint,Cinv)

# Cinv = np.ones(dvec.shape)


file_name = 'data/sim_gg.json'

save_dict = {'z':zz,'space':'rsd','kmin':kmin,'kmax':kmax,'Cov':Cov.tolist(),'Cinv':Cinv.tolist(),
             'd_pkmu':d_pkmu.tolist(),'k':sim_k.tolist(),'mu':mu.tolist(),'d_ell':dd.tolist()}
# Save array to JSON file
with open(file_name, 'w') as json_file:
    json.dump(save_dict, json_file)

[12.0, 12.7, 0.66, 1, 0.33, 0, 0, 0, 0, 0, 1.0, 1.0, 1.0, 1.0, 1.0]
(7500,) (7500,)


In [10]:

# ff = open('/pscratch/sd/h/hebina/AbacusLBG/ibis_tertiary44/LAE_auto/lae_clustering_concat_c000_ph100_s_zcv.json')
ff = open('/pscratch/sd/h/hebina/AbacusLBG/ibis_tertiary44/LAE_auto_v2/hod/lae_clustering_c000_ph100_z3.0_r_zcv.json')
# ff = open(self.datfn)
simdata = json.load(ff)
sim_k = np.array(simdata['k'])
k = np.array(simdata['k'])
Nk = len(k)
Nmu = 50
mu = np.linspace(0.,1.,Nmu)
MU = np.tile(mu,Nk)
dmu = list(mu[1:]-mu[:-1])
dmu.append(dmu[-1])
dmu = np.array(dmu)

dk = list(k[1:]-k[:-1])
dk.insert(0,dk[0])
dk = np.array(dk)
k = np.repeat(k,Nmu)
dk = np.repeat(dk,Nmu)
dmu = np.tile(dmu,Nk)

print(simdata['mocks'][ihod]['hod'])
dd = [simdata['mocks'][ihod]['pk0'],simdata['mocks'][ihod]['pk2']]
dd = np.array(dd).T

true_SN = 1e9/min(simdata['mocks'][ihod]['nobj'],1e7)

dvec = np.array([np.repeat(dd[:,0],Nmu),np.repeat(dd[:,1],Nmu)])
d_pkmu = dvec[0]+0.5*(3*MU**2-1)*dvec[1]
d_pkmu -= true_SN
print(k.shape,d_pkmu.shape)
Vsurvey = (1e3)**3

prefactor = (2.*np.pi**2.) / (dk*dmu*Vsurvey*k**2.) * k**kpow
Cov = prefactor*2*d_pkmu**2
# constraints = forecast1.compute_wedge(z,kmin=kmin)*forecast1.kmax_constraint(z,kmax_knl)
# constraints *= (forecast1.k > kmin)
knl  = 0.5
kmin = 0.003
kmax = knl


k_constraint = (k>kmin)*(k<kmax)
Cov += 1e-20
Cinv = 1/Cov
Cinv *= k_constraint
# Cinv = np.einsum('i,j,ij->ij',k_constraint,k_constraint,Cinv)

# Cinv = np.ones(dvec.shape)


file_name = 'data/sim_gg_real.json'

save_dict = {'z':zz,'space':'rsd','kmin':kmin,'kmax':kmax,'Cov':Cov.tolist(),'Cinv':Cinv.tolist(),
             'd_pkmu':d_pkmu.tolist(),'k':sim_k.tolist(),'mu':mu.tolist(),'d_ell':dd.tolist()}
# Save array to JSON file
with open(file_name, 'w') as json_file:
    json.dump(save_dict, json_file)

[12.0, 12.7, 0.66, 1, 0.33, 0, 0, 0, 0, 0, 1.0, 1.0, 1.0, 1.0, 1.0]
(7500,) (7500,)


In [11]:

# ff = open('/pscratch/sd/h/hebina/AbacusLBG/ibis_tertiary44/LAE_auto/lae_clustering_concat_c000_ph100_s_zcv.json')
ff = open('/pscratch/sd/h/hebina/AbacusLBG/ibis_tertiary44/LAE_auto_v2/hod/lae_clustering_c000_ph100_z3.0_r_gm.json')
# ff = open(self.datfn)
simdata = json.load(ff)
sim_k = np.array(simdata['k'])
k = np.array(simdata['k'])
Nk = len(k)
Nmu = 50
mu = np.linspace(0.,1.,Nmu)
MU = np.tile(mu,Nk)
dmu = list(mu[1:]-mu[:-1])
dmu.append(dmu[-1])
dmu = np.array(dmu)

dk = list(k[1:]-k[:-1])
dk.insert(0,dk[0])
dk = np.array(dk)
k = np.repeat(k,Nmu)
dk = np.repeat(dk,Nmu)
dmu = np.tile(dmu,Nk)

print(simdata['mocks'][ihod]['hod'])
dd = [simdata['mocks'][ihod]['pk0'],simdata['mocks'][ihod]['pk2']]
dd = np.array(dd).T

# dvec = np.array([np.repeat(dd[:,0],Nmu),np.repeat(dd[:,1],Nmu),np.repeat(dd[:,2],Nmu)])
# d_pkmu = dvec[0]+0.5*(3*MU**2-1)*dvec[1]+0.125*(35*MU**4-30*MU**2+3)*dvec[2]
dvec = np.array([np.repeat(dd[:,0],Nmu),np.repeat(dd[:,1],Nmu)])
d_pkmu = dvec[0]+0.5*(3*MU**2-1)*dvec[1]
print(k.shape,d_pkmu.shape)
Vsurvey = (1e3)**3

prefactor = (2.*np.pi**2.) / (dk*dmu*Vsurvey*k**2.) * k**kpow
Cov = prefactor*2*d_pkmu**2
# constraints = forecast1.compute_wedge(z,kmin=kmin)*forecast1.kmax_constraint(z,kmax_knl)
# constraints *= (forecast1.k > kmin)
knl  = 0.5
kmin = 0.003
kmax = knl


k_constraint = (k>kmin)*(k<kmax)
Cov += 1e-20
Cinv = 1/Cov
Cinv *= k_constraint
# Cinv = np.einsum('i,j,ij->ij',k_constraint,k_constraint,Cinv)

# Cinv = np.ones(dvec.shape)

file_name = 'data/sim_gm.json'

save_dict = {'z':zz,'space':'rsd','kmin':kmin,'kmax':kmax,'Cov':Cov.tolist(),'Cinv':Cinv.tolist(),
             'd_pkmu':d_pkmu.tolist(),'k':sim_k.tolist(),'mu':mu.tolist(),'d_ell':dd.tolist()}
# Save array to JSON file
with open(file_name, 'w') as json_file:
    json.dump(save_dict, json_file)

[12.0, 12.7, 0.66, 1, 0.33, 0, 0, 0, 0, 0, 1.0, 1.0, 1.0, 1.0, 1.0]
(2500,) (2500,)


In [12]:

# ff = open('/pscratch/sd/h/hebina/AbacusLBG/ibis_tertiary44/LAE_auto/lae_clustering_concat_c000_ph100_s_zcv.json')
ff = open('/pscratch/sd/h/hebina/AbacusLBG/ibis_tertiary44/LAE_auto_v2/hod/lae_clustering_c000_ph100_z3.0_r_mm.json')
# ff = open(self.datfn)
simdata = json.load(ff)
sim_k = np.array(simdata['k'])
k = np.array(simdata['k'])
Nk = len(k)
Nmu = 50
mu = np.linspace(0.,1.,Nmu)
MU = np.tile(mu,Nk)
dmu = list(mu[1:]-mu[:-1])
dmu.append(dmu[-1])
dmu = np.array(dmu)

dk = list(k[1:]-k[:-1])
dk.insert(0,dk[0])
dk = np.array(dk)
k = np.repeat(k,Nmu)
dk = np.repeat(dk,Nmu)
dmu = np.tile(dmu,Nk)

ihod = 0
print(simdata['mocks'][ihod]['hod'])
dd = [simdata['mocks'][ihod]['pk0'],simdata['mocks'][ihod]['pk2']]
dd = np.array(dd).T

dvec = np.array([np.repeat(dd[:,0],Nmu),np.repeat(dd[:,1],Nmu)])
d_pkmu = dvec[0]+0.5*(3*MU**2-1)*dvec[1]

d_pkmu -= mSN[1]

print(k.shape,d_pkmu.shape)
Vsurvey = (1e3)**3

prefactor = (2.*np.pi**2.) / (dk*dmu*Vsurvey*k**2.) * k**kpow
Cov = prefactor*2*d_pkmu**2
# constraints = forecast1.compute_wedge(z,kmin=kmin)*forecast1.kmax_constraint(z,kmax_knl)
# constraints *= (forecast1.k > kmin)
knl  = 0.5
kmin = 0.003
kmax = knl


k_constraint = (k>kmin)*(k<kmax)
Cov += 1e-20
Cinv = 1/Cov
Cinv *= k_constraint
# Cinv = np.einsum('i,j,ij->ij',k_constraint,k_constraint,Cinv)

# Cinv = np.ones(dvec.shape)

file_name = 'data/sim_mm.json'

save_dict = {'z':zz,'space':'rsd','kmin':kmin,'kmax':kmax,'Cov':Cov.tolist(),'Cinv':Cinv.tolist(),
             'd_pkmu':d_pkmu.tolist(),'k':sim_k.tolist(),'mu':mu.tolist(),'d_ell':dd.tolist()}
# Save array to JSON file
with open(file_name, 'w') as json_file:
    json.dump(save_dict, json_file)

[11.25, 11.948970004336019, 0.66, 1, 0.66, 0, 0, 0, 0, 0, 1.0, 1.0, 1.0, 1.0, 1.0]
(2500,) (2500,)


# $z=2.5$

In [ ]:
zz = 2.5
ihod = 1
kpow = 1 # manual adjustment to covariance
kpow = 1.5 # manual adjustment to covariance

In [ ]:

# ff = open('/pscratch/sd/h/hebina/AbacusLBG/ibis_tertiary44/LAE_auto/lae_clustering_concat_c000_ph100_s_zcv.json')
ff = open('/pscratch/sd/h/hebina/AbacusLBG/ibis_tertiary44/LAE_auto_v2/hod/lae_clustering_c000_ph100_z2.5_s_zcv.json')
# ff = open(self.datfn)
simdata = json.load(ff)
sim_k = np.array(simdata['k'])
k = np.array(simdata['k'])
Nk = len(k)
Nmu = 50
mu = np.linspace(0.,1.,Nmu)
MU = np.tile(mu,Nk)
dmu = list(mu[1:]-mu[:-1])
dmu.append(dmu[-1])
dmu = np.array(dmu)

dk = list(k[1:]-k[:-1])
dk.insert(0,dk[0])
dk = np.array(dk)
k = np.repeat(k,Nmu)
dk = np.repeat(dk,Nmu)
dmu = np.tile(dmu,Nk)

print(simdata['mocks'][ihod]['hod'])
dd = [simdata['mocks'][ihod]['pk0'],simdata['mocks'][ihod]['pk2']]
dd = np.array(dd).T

# take out shot noise
true_SN = 1e9/min(simdata['mocks'][ihod]['nobj'],1e7)

dvec = np.array([np.repeat(dd[:,0],Nmu),np.repeat(dd[:,1],Nmu)])
d_pkmu = dvec[0]+0.5*(3*MU**2-1)*dvec[1]
d_pkmu -= true_SN
print(k.shape,d_pkmu.shape)
Vsurvey = (1e3)**3

prefactor = (2.*np.pi**2.) / (dk*dmu*Vsurvey*k**2.) * k**kpow
Cov = prefactor*2*d_pkmu**2
# constraints = forecast1.compute_wedge(z,kmin=kmin)*forecast1.kmax_constraint(z,kmax_knl)
# constraints *= (forecast1.k > kmin)
knl  = 0.5
kmin = 0.003
kmax = knl


k_constraint = (k>kmin)*(k<kmax)
Cov += 1e-20
Cinv = 1/Cov
Cinv *= k_constraint
# Cinv = np.einsum('i,j,ij->ij',k_constraint,k_constraint,Cinv)

# Cinv = np.ones(dvec.shape)


file_name = 'data/sim_gg.json'

save_dict = {'z':zz,'space':'rsd','kmin':kmin,'kmax':kmax,'Cov':Cov.tolist(),'Cinv':Cinv.tolist(),
             'd_pkmu':d_pkmu.tolist(),'k':sim_k.tolist(),'mu':mu.tolist(),'d_ell':dd.tolist()}
# Save array to JSON file
with open(file_name, 'w') as json_file:
    json.dump(save_dict, json_file)

In [ ]:

# ff = open('/pscratch/sd/h/hebina/AbacusLBG/ibis_tertiary44/LAE_auto/lae_clustering_concat_c000_ph100_s_zcv.json')
ff = open('/pscratch/sd/h/hebina/AbacusLBG/ibis_tertiary44/LAE_auto_v2/hod/lae_clustering_c000_ph100_z2.5_r_zcv.json')
# ff = open(self.datfn)
simdata = json.load(ff)
sim_k = np.array(simdata['k'])
k = np.array(simdata['k'])
Nk = len(k)
Nmu = 50
mu = np.linspace(0.,1.,Nmu)
MU = np.tile(mu,Nk)
dmu = list(mu[1:]-mu[:-1])
dmu.append(dmu[-1])
dmu = np.array(dmu)

dk = list(k[1:]-k[:-1])
dk.insert(0,dk[0])
dk = np.array(dk)
k = np.repeat(k,Nmu)
dk = np.repeat(dk,Nmu)
dmu = np.tile(dmu,Nk)

print(simdata['mocks'][ihod]['hod'])
dd = [simdata['mocks'][ihod]['pk0'],simdata['mocks'][ihod]['pk2']]
dd = np.array(dd).T

# take out shot noise
true_SN = 1e9/min(simdata['mocks'][ihod]['nobj'],1e7)

dvec = np.array([np.repeat(dd[:,0],Nmu),np.repeat(dd[:,1],Nmu)])
d_pkmu = dvec[0]+0.5*(3*MU**2-1)*dvec[1]
d_pkmu -= true_SN
print(k.shape,d_pkmu.shape)
Vsurvey = (1e3)**3

prefactor = (2.*np.pi**2.) / (dk*dmu*Vsurvey*k**2.) * k**kpow
Cov = prefactor*2*d_pkmu**2
# constraints = forecast1.compute_wedge(z,kmin=kmin)*forecast1.kmax_constraint(z,kmax_knl)
# constraints *= (forecast1.k > kmin)
knl  = 0.5
kmin = 0.003
kmax = knl


k_constraint = (k>kmin)*(k<kmax)
Cov += 1e-20
Cinv = 1/Cov
Cinv *= k_constraint
# Cinv = np.einsum('i,j,ij->ij',k_constraint,k_constraint,Cinv)

# Cinv = np.ones(dvec.shape)


file_name = 'data/sim_gg_real.json'

save_dict = {'z':zz,'space':'rsd','kmin':kmin,'kmax':kmax,'Cov':Cov.tolist(),'Cinv':Cinv.tolist(),
             'd_pkmu':d_pkmu.tolist(),'k':sim_k.tolist(),'mu':mu.tolist(),'d_ell':dd.tolist()}
# Save array to JSON file
with open(file_name, 'w') as json_file:
    json.dump(save_dict, json_file)

In [ ]:

# ff = open('/pscratch/sd/h/hebina/AbacusLBG/ibis_tertiary44/LAE_auto/lae_clustering_concat_c000_ph100_s_zcv.json')
ff = open('/pscratch/sd/h/hebina/AbacusLBG/ibis_tertiary44/LAE_auto_v2/hod/lae_clustering_c000_ph100_z2.5_r_gm.json')
# ff = open(self.datfn)
simdata = json.load(ff)
sim_k = np.array(simdata['k'])
k = np.array(simdata['k'])
Nk = len(k)
Nmu = 50
mu = np.linspace(0.,1.,Nmu)
MU = np.tile(mu,Nk)
dmu = list(mu[1:]-mu[:-1])
dmu.append(dmu[-1])
dmu = np.array(dmu)

dk = list(k[1:]-k[:-1])
dk.insert(0,dk[0])
dk = np.array(dk)
k = np.repeat(k,Nmu)
dk = np.repeat(dk,Nmu)
dmu = np.tile(dmu,Nk)

print(simdata['mocks'][ihod]['hod'])
dd = [simdata['mocks'][ihod]['pk0'],simdata['mocks'][ihod]['pk2']]
dd = np.array(dd).T

# dvec = np.array([np.repeat(dd[:,0],Nmu),np.repeat(dd[:,1],Nmu),np.repeat(dd[:,2],Nmu)])
# d_pkmu = dvec[0]+0.5*(3*MU**2-1)*dvec[1]+0.125*(35*MU**4-30*MU**2+3)*dvec[2]
dvec = np.array([np.repeat(dd[:,0],Nmu),np.repeat(dd[:,1],Nmu)])
d_pkmu = dvec[0]+0.5*(3*MU**2-1)*dvec[1]
print(k.shape,d_pkmu.shape)
Vsurvey = (1e3)**3

prefactor = (2.*np.pi**2.) / (dk*dmu*Vsurvey*k**2.) * k**kpow
Cov = prefactor*2*d_pkmu**2
# constraints = forecast1.compute_wedge(z,kmin=kmin)*forecast1.kmax_constraint(z,kmax_knl)
# constraints *= (forecast1.k > kmin)
knl  = 0.5
kmin = 0.003
kmax = knl


k_constraint = (k>kmin)*(k<kmax)
Cov += 1e-20
Cinv = 1/Cov
Cinv *= k_constraint
# Cinv = np.einsum('i,j,ij->ij',k_constraint,k_constraint,Cinv)

# Cinv = np.ones(dvec.shape)

file_name = 'data/sim_gm.json'

save_dict = {'z':zz,'space':'rsd','kmin':kmin,'kmax':kmax,'Cov':Cov.tolist(),'Cinv':Cinv.tolist(),
             'd_pkmu':d_pkmu.tolist(),'k':sim_k.tolist(),'mu':mu.tolist(),'d_ell':dd.tolist()}
# Save array to JSON file
with open(file_name, 'w') as json_file:
    json.dump(save_dict, json_file)

In [ ]:

# ff = open('/pscratch/sd/h/hebina/AbacusLBG/ibis_tertiary44/LAE_auto/lae_clustering_concat_c000_ph100_s_zcv.json')
ff = open('/pscratch/sd/h/hebina/AbacusLBG/ibis_tertiary44/LAE_auto_v2/hod/lae_clustering_c000_ph100_z2.5_r_mm.json')
# ff = open(self.datfn)
simdata = json.load(ff)
sim_k = np.array(simdata['k'])
k = np.array(simdata['k'])
Nk = len(k)
Nmu = 50
mu = np.linspace(0.,1.,Nmu)
MU = np.tile(mu,Nk)
dmu = list(mu[1:]-mu[:-1])
dmu.append(dmu[-1])
dmu = np.array(dmu)

dk = list(k[1:]-k[:-1])
dk.insert(0,dk[0])
dk = np.array(dk)
k = np.repeat(k,Nmu)
dk = np.repeat(dk,Nmu)
dmu = np.tile(dmu,Nk)

ihod = 0
print(simdata['mocks'][ihod]['hod'])
dd = [simdata['mocks'][ihod]['pk0'],simdata['mocks'][ihod]['pk2']]
dd = np.array(dd).T

# dvec = np.array([np.repeat(dd[:,0],Nmu),np.repeat(dd[:,1],Nmu),np.repeat(dd[:,2],Nmu)])
# d_pkmu = dvec[0]+0.5*(3*MU**2-1)*dvec[1]+0.125*(35*MU**4-30*MU**2+3)*dvec[2]
dvec = np.array([np.repeat(dd[:,0],Nmu),np.repeat(dd[:,1],Nmu)])
d_pkmu = dvec[0]+0.5*(3*MU**2-1)*dvec[1]
d_pkmu -= mSN[0]
print(k.shape,d_pkmu.shape)
Vsurvey = (1e3)**3

prefactor = (2.*np.pi**2.) / (dk*dmu*Vsurvey*k**2.) * k**kpow
Cov = prefactor*2*d_pkmu**2
# constraints = forecast1.compute_wedge(z,kmin=kmin)*forecast1.kmax_constraint(z,kmax_knl)
# constraints *= (forecast1.k > kmin)
knl  = 0.5
kmin = 0.003
kmax = knl


k_constraint = (k>kmin)*(k<kmax)
Cov += 1e-20
Cinv = 1/Cov
Cinv *= k_constraint
# Cinv = np.einsum('i,j,ij->ij',k_constraint,k_constraint,Cinv)

# Cinv = np.ones(dvec.shape)

file_name = 'data/sim_mm.json'

save_dict = {'z':zz,'space':'rsd','kmin':kmin,'kmax':kmax,'Cov':Cov.tolist(),'Cinv':Cinv.tolist(),
             'd_pkmu':d_pkmu.tolist(),'k':sim_k.tolist(),'mu':mu.tolist(),'d_ell':dd.tolist()}
# Save array to JSON file
with open(file_name, 'w') as json_file:
    json.dump(save_dict, json_file)